In [1]:
import requests
import urllib.request
import pandas as pd
import csv
from bs4 import BeautifulSoup
import pickle

import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import re

In [2]:
df_geomet = pd.read_csv(os.path.abspath('../data/Processed/Cities_imputed.csv'), index_col=0).loc[:,['City', 'Country', 'Latitude', 'Longitude']]
df_geomet

,City,Country,Latitude,Longitude
284,Baltimore(MD),United States,39.290882,-76.610759
9,Melbourne,Australia,-37.814218,144.963161
185,Niamey,Niger,13.524834,2.109823
327,Hanoi,Vietnam,21.029450,105.854444
66,Urumqi,China,43.419754,87.319461
...,...,...,...,...
225,Durban,South Africa,-29.861825,31.009909
103,Kumasi,Ghana,6.698081,-1.623040
250,Izmir,Turkey,38.414733,27.143412
249,Ankara,Turkey,39.716044,32.705995


## Distance to equator

Latitude is measured as a degree between between -90 to 90 from Antartic to Artic. Using the radius of the Earth which is roughly 6371 km the distance from the equator can be added which is done as a new feature.

In [3]:
df_geomet["Dist_Equator"] = np.abs(df_geomet["Latitude"]*np.pi/180*6371)

## Elevation above ground
It was also desired to use the elevation of the city. For this an api, called *elevation-api* was found. In the free version the resultion was limited to 5000m but we purchased a key to get it down to 90m resolution which is deemed sufficient as it is cities that is worked with:

In [4]:
# It costs me 0.03$ to run this cell so please run with care
def get_elevation(lat, long):
    query = f"https://elevation-api.io/api/elevation?points=({lat},{long})&resolution=90&key=a-a322u40-98wfe6g9a94--JJcG8P-"
    r = requests.get(query).json()  # json object, various ways you can extract value
            # one approach is to use pandas json functionality:
    elevation = r['elevations'][0]['elevation']
    return elevation

df_geomet['Elevation'] = [get_elevation(x,y) for x,y in zip(df_geomet['Latitude'], df_geomet['Longitude'])]

## Distance to coast

Another geological feature that is deemed insteresting is the distance to the coast. Initially it was desired to use an api for this but none were found at a *fair* price. Instead hardcoded data from NASA is used. This data is publicly avialable at: https://oceancolor.gsfc.nasa.gov/docs/distfromcoast/. Based on longitude and latitude (rounded to the nearest 0.04) the distance to the coast can be loked up. The data is converted to a 2d-dataframe to allow for quicker lookup. The resulting dataframe is shown below:

In [5]:
dist_to_coast_df = pd.pivot_table(pd.read_table('../data/Raw/dist2coast.txt', header = None), index = 0, columns = 1, values = 2)
dist_to_coast_df.head()

1,-89.98,-89.94,-89.90,-89.86,-89.82,-89.78,-89.74,-89.70,-89.66,-89.62,...,89.62,89.66,89.70,89.74,89.78,89.82,89.86,89.90,89.94,89.98
0,,,,,,,,,,,,,,,,,,,,,
-179.98,1274.3,1269.99,1265.69,1261.39,1257.09,1252.79,1248.49,1244.20,1239.90,1235.61,...,746.661,742.882,739.111,735.348,731.592,727.844,724.104,720.373,716.650,712.935
-179.94,1274.3,1269.99,1265.69,1261.39,1257.09,1252.79,1248.49,1244.19,1239.90,1235.60,...,746.646,742.868,739.099,735.337,731.582,727.836,724.098,720.368,716.647,712.934
-179.90,1274.3,1269.99,1265.69,1261.38,1257.08,1252.78,1248.48,1244.18,1239.89,1235.59,...,746.630,742.854,739.086,735.326,731.573,727.829,724.092,720.364,716.644,712.933
-179.86,1274.3,1269.99,1265.69,1261.38,1257.08,1252.78,1248.48,1244.18,1239.88,1235.58,...,746.614,742.840,739.074,735.315,731.564,727.821,724.086,720.360,716.642,712.932
-179.82,1274.3,1269.99,1265.68,1261.38,1257.08,1252.77,1248.47,1244.17,1239.87,1235.58,...,746.598,742.825,739.061,735.304,731.554,727.813,724.080,720.355,716.639,712.932


Now all longitudes and latitudes has to be rounded to the nearest 0.04. Notice that the fractions are offset by 0.02, so it goes xx.02, xx.06$\dots$. A function to round a number to match the NASA data is implemented below:

In [6]:
def round04(x):
    return round(x - ((x % 0.04) - 0.02), 2)

Using this function and the NASA data all distances to the coast can be added as it is done below

In [7]:
df_geomet['Dist_to_coast'] = [round(dist_to_coast_df.loc[long,lat]) for long,lat in zip([round04(x) for x in df_geomet.Longitude], [round04(x) for x in df_geomet.Latitude])]

With these three *geographic*

In [8]:
df_geomet

,City,Country,Latitude,Longitude,Dist_Equator,Elevation,Dist_to_coast
284,Baltimore(MD),United States,39.290882,-76.610759,4368.946697,6.0,2
9,Melbourne,Australia,-37.814218,144.963161,4204.749152,25.0,4
185,Niamey,Niger,13.524834,2.109823,1503.892925,209.0,781
327,Hanoi,Vietnam,21.029450,105.854444,2338.368128,19.0,77
66,Urumqi,China,43.419754,87.319461,4828.056361,2071.0,2312
...,...,...,...,...,...,...,...
225,Durban,South Africa,-29.861825,31.009909,3320.483440,13.0,2
103,Kumasi,Ghana,6.698081,-1.623040,744.792625,261.0,177
250,Izmir,Turkey,38.414733,27.143412,4271.523429,126.0,1
249,Ankara,Turkey,39.716044,32.705995,4416.222588,1069.0,197


## Meteorologic data

With the geographic data in place meteorologic data is also added. 

Most cities in the data set have very informative Wikipedia Sites that data that is publicaly available to anyone with a internet connection. As we're are particular interested in the comparing the CO2 emision with climate/geological data we're partilcuar interested in the section on each 
city's Wikipedia page that describe a city's cliamte.  Below can a screenshot be seen of the Climate Data table for Berlin. 

![title](figures/Berlin_climate.png)

Similar tables exisist for almost all the cities in the dataset on Wikipedia. We decided to on Wikipedia data as it was difficult to find an open API which could return climate data about each city whereas Wikipedia is open to everyone. Getting every Cliamte Table turned out to be a much more cumbersome task than first expexted. It turned out that though it appear as almost every city has "same" Climate Table like one above they variy and their format is not conscient. This section of the Notebook will explain how the data was scraped and processed such that it could be used as modelling variable. The Webscraping part heabily depends on the `request` and `BeautifulSoup` libraries. The two libraries makes it easy to scrape information from web pages. The first establish a connection to the webpage and the latter handles the scraping. 


## Setting up the Wikipedia links
The city names does not match one-to-one with their respective Wikipedia urls. Most can be easily fixed by removing letters inside parethesis and replacing white spaces with underscores (_). E.g. the city Baltimore located in the American state Marylan has the name  *Baltimore(MD)* in the data but its Wikipedia url just contains *Baltimore*. Likewise the South African captial *Cape Town* Wikipedia page is found with url *Cape_Town*. These two quick fixes can be achieved using regular expressions that comes with in buildt library `re`. This is done in the cell below:

In [9]:
df_geomet['wiki_link'] = [re.sub(r'\([^)]*\)', '', city).replace(" ", "_") for city in df_geomet.City]
df_geomet.loc[df_geomet['wiki_link'] == 'Birmingham']

,City,Country,Latitude,Longitude,Dist_Equator,Elevation,Dist_to_coast,wiki_link
319,Birmingham(AL),United States,33.570499,-86.765783,3732.869174,207.0,331,Birmingham
261,Birmingham,United Kingdom,52.479699,-1.902691,5835.476303,153.0,80,Birmingham


Not all cities wikiepedia URL is fixed with the above regular expressions. Unfortunally quite some cities need manuel URL change. This have primaily been due to that the city name refers to two cities located next to eachother like *Rotterdam-Hague* as well as the city name exist in multiple countries. This have been fixing by choosing on the cities under the assumption that they are located close to eachother and thus have *very* similar climate and by specifiying the URL for city name that are in multiple countries. The cells below ensure that the *right* wikiepedia link is established.

In [10]:
def update_wiki_link(df, city, city_update):
    if city in df.wiki_link.values:
        df.loc[df['wiki_link'] == city, 'wiki_link'] = city_update
    else:
        print(city)

In [11]:
cities_updates = [
    ('Santa_Cruz', 'Santa_Cruz_(Bolivia)'),
    ('Cordoba' , 'Córdoba,_Argentina'),
    ('San_Francisco_Bay_Area', 'San_Fransisco'),
    ('Tampa-St._Petersburg', 'Saint_Petersburg_(Florida)'),
    ('Memphis', 'Memphis,_Tennessee'),
    ('Denver-Aurora', 'Aurora,_Colorado'),
    ('Rotterdam-Hague', 'Rotterdam'),
    ('Nashville-Davidson', 'Nashville,_Tennessee'),
    ('Cologne-Bonn', 'Cologne'),
    ('Richmond', 'Richmond,_Virginia'),
    ('Rochester', 'Rochester,_New_York'),
    ('Phoenix-Mesa', 'Phoenix_(Arizona)'),
    ('Leon', 'León,_Guanajuato'),
    ('Buffalo', 'Buffalo,_New_York'),
    ('Allentown-Bethlehem', 'Allentown,_Pennsylvania'),
    ('San_Juan', 'San_Juan,_Puerto_Rico'),
    ('Puebla', 'Puebla_(city)'),
    ('San_Jose', 'San_José,_Costa_Rica'),
    ('Dallas-Fort_Worth', 'Fort_Worth,_Texas'),
    ('Seoul-Incheon', 'Seoul'),
    ('Lome', 'Lomé'),
    ('Chihuahua', 'Chihuahua_City'),
    ('Fukuoka-Kitakyushu', 'Fukuoka'),
    ('Davao', 'Davao_City'),
    ('New_York', 'New_York_City'),
    ('Osaka-Kobe-Kyoto', 'Osaka'),
    ('Kansas_City', 'Kansas_City,_Missouri'),
    ('Minneapolis-St._Paul', 'Minneapolis'),
    ('Portland', 'Portland,_Oregon'),
    ('Phoenix_(Arizona)', 'Phoenix,_Arizona'),
    ('Washington', 'Washington,_D.C.'),
    ('Columbus', 'Columbus,_Ohio'),
    ('Salvador', 'Salvador,_Bahia'),
    ('Xi‚Äôan', 'Xi%27an'),
    ('Concepcion', 'Concepción,_Chile'),
    ('Providence', 'Providence,_Rhode_Island')
]

for city, city_update in cities_updates:
    update_wiki_link(
        df = df_geomet,
        city = city,
        city_update = city_update
    )

The city name *Birmingham* and *Valencia* appear twice in the dataset. The reason is that there is Burmingham in the United Kingdom as well as in the United States in Alabama. Similar Valencia is a city in Spain as well as a city in the Venzuela. The cell below ensure that the appropriate wikipedia link exist for all four cities.

In [12]:
df_geomet.loc[(df_geomet['wiki_link'] == 'Birmingham') & (df_geomet['Country'] == 'United States')] = 'Birmingham,_Alabama'
df_geomet.loc[(df_geomet['wiki_link'] == 'Valencia') & (df_geomet['Country'] == 'United Venezuela')] = 'Valencia,_Carabobo'

## Extracting the the Climate Data with Beutiful Soup

To exstract the cliamte tables three function have been created.

1. `extract_wiki_page` this function establish a connection to the selected Wikipdia Page. Using BeutifulSoup it exstact the entire page in raw HTML format.
1. `extract_climate_table` this function process the BeutifulSoup object that contain the entire Wikipdia page of selected City. If a climate table exists on the page it select only this part of the page otherwise it returns `None`.
1. `process_climate_table` the last function process the selected climate table such that the HTML is converted into a Pandas DataFrame object. That makes it a lot easier to work with.

The three function can be seen in the cells below.

In [13]:
def extract_wiki_page(wiki_url):
    response = requests.get(wiki_url)
    status_code = response.status_code

    soup = False
    if status_code == 200:
         soup = BeautifulSoup(response.text,"html.parser")

    return soup


def extract_climate_table(soup):
    tables = soup.findAll('table',{"class":"collapsible"})
    
    # Exstract the table with the Cliamte Data if it exist
    select_table = None
    for table in tables:
        header = table.findAll(['th'])[0].getText()[0:12].lower()
        
        if 'climate' in header:
            select_table = table
            break
    
    return select_table



def process_climate_table(table):
    trs = table.findAll(['tr'])[1:]

    for i, cell in enumerate(trs):
        th = cell.find_all('th')
        th_data = [col.text.strip('\n') for col in th]  
        td = cell.find_all('td')
        row = [i.text.replace('\n','') for i in td]

        if i == 0:
            columns = th_data
            city_climate_df_tmp = pd.DataFrame(columns=columns)
        
        elif len(th_data) == 0:
            continue
        else:    
            tmp = pd.DataFrame({key: [value] for key, value in zip(columns, th_data+row)})
            city_climate_df_tmp = city_climate_df_tmp.append(tmp)
    
    city_climate_df_tmp.rename(columns={"Month": "Climate_variable"}, inplace = True)
    city_climate_df_tmp.set_index('Climate_variable', inplace = True)

    return city_climate_df_tmp

Below can the power of the three functions be seen. Here the city of Berlin cliamte data is extracted and returned in the form of Pandas Dataframe. 

In [14]:
url = 'https://en.wikipedia.org/wiki/' + 'Berlin'
soup = extract_wiki_page(url)
city_climate_table = extract_climate_table(soup)
process_climate_table(city_climate_table)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Record high °C (°F),15.1(59.2),18.0(64.4),25.8(78.4),30.8(87.4),32.7(90.9),35.4(95.7),37.3(99.1),38.0(100.4),32.3(90.1),27.7(81.9),20.4(68.7),15.6(60.1),38.0(100.4)
Average high °C (°F),2.8(37.0),4.3(39.7),8.7(47.7),14.3(57.7),19.4(66.9),22.0(71.6),24.6(76.3),24.2(75.6),19.3(66.7),13.8(56.8),7.3(45.1),3.3(37.9),13.7(56.7)
Daily mean °C (°F),0.1(32.2),0.9(33.6),4.3(39.7),9.0(48.2),14.0(57.2),16.8(62.2),19.1(66.4),18.5(65.3),14.2(57.6),9.4(48.9),4.4(39.9),1.0(33.8),9.3(48.7)
Average low °C (°F),−2.8(27.0),−2.4(27.7),0.4(32.7),3.5(38.3),8.2(46.8),11.2(52.2),13.5(56.3),13.0(55.4),9.6(49.3),5.4(41.7),1.4(34.5),−1.6(29.1),5.0(41.0)
Record low °C (°F),−25.3(−13.5),−22.0(−7.6),−16.0(3.2),−7.4(18.7),−2.8(27.0),1.3(34.3),4.9(40.8),4.6(40.3),−0.9(30.4),−7.7(18.1),−12.0(10.4),−24.0(−11.2),−25.3(−13.5)
Average precipitation mm (inches),37.2(1.46),30.1(1.19),39.3(1.55),33.7(1.33),52.6(2.07),60.2(2.37),52.5(2.07),53.0(2.09),39.5(1.56),32.2(1.27),37.8(1.49),46.1(1.81),515.2(20.28)
Mean monthly sunshine hours,57.6,71.5,119.4,191.2,229.6,230.0,232.4,217.3,162.3,114.7,54.9,46.9,"1,727.6"
Average ultraviolet index,1,1,2,4,5,6,6,5,4,2,1,0,3


### Extracting all Cities' Climate Data
Now all cities will be processed. Each dataframe will be stored in a dictionay where the key is the Wikipeida enterpeation of the cities name. 

In [20]:
climate_data_dict = dict()

for wiki_link in df_geomet.wiki_link:
    url = 'https://en.wikipedia.org/wiki/' + wiki_link
    soup = extract_wiki_page(url)

    if soup:
        city_climate_table = extract_climate_table(soup)

        # If the climate table is available on Wikipedia
        if city_climate_table:
            df_city_climate = process_climate_table(city_climate_table)
            climate_data_dict[wiki_link] = df_city_climate

        # If the wikipedia page does not have climate table
        else: 
            climate_data_dict[wiki_link] = None
            print('No climate data available for city: ', wiki_link)
    else:
        print('Could not find citys: ', wiki_link, 'Wikipedia page')

No climate data available for city:  Cebu
No climate data available for city:  Caracas
No climate data available for city:  Santo_Domingo
No climate data available for city:  Monterrey


As can be seen all cities cliamte table have been exstracted except the cities of `Cebu`, `Caracas`, `Santo_Domingo`, and `Monterrey`. The city of Cebu simply does not have a Climate Table so there is nothing to exstract. However the three remaining cities *does* have a wikipedia page with such climate table. However the format is much different from the other cities so it does not work. Rather than tweeking things for hours we decided to let the cities be. Instead all will be imputed in the end of the notebook.

As each each city takes a few seconds to scrape, primarily due to the repsonse time of wikipedia, the dictionary that contains a Pandas dataframe with cliamte data for each have been saved. Here the library `pickle` habe been used. “Pickling” is the process whereby a Python object is converted into a byte stream, and “unpickling” is the inverse operation, whereby a byte stream (from a binary file or bytes-like object) is converted back into a python object again.

In [26]:
# Save the climate data in pickle as it take quite some time to geneate.
with open('climate_data_dict.pickle', 'wb') as handle:
    pickle.dump(climate_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Processing the exstraced Climate Data
As mentioned each Wikipedia Page climate apge, though they look similar, are different.  This is easily proven by examining two different cities. In the cells below are the dataframe for city of `Austin` and `Freetown` showed. Notice the different names of the climate variables. One prefers Farhenheit over Celcius. One mentions snowfall where the other does not. In order to use this data the data needs to be on the same scale and it need to be same variable.

However, out the 69 different variable, many represents the same thing with a different name, as well many variable which occur many time overall. This is examined in the cell below:

In [28]:
with open('climate_data_dict.pickle', 'rb') as handle:
    climate_data_dict = pickle.load(handle)
climate_data_dict['Austin']

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Record high °F (°C),90(32),99(37),98(37),99(37),104(40),109(43),109(43),112(44),112(44),100(38),91(33),90(32),112(44)
Mean maximum °F (°C),79.3(26.3),83.3(28.5),86.6(30.3),91.7(33.2),95.4(35.2),98.1(36.7),101.0(38.3),102.9(39.4),99.1(37.3),93.3(34.1),84.8(29.3),79.7(26.5),103.9(39.9)
Average high °F (°C),61.5(16.4),65.2(18.4),72.2(22.3),79.8(26.6),86.5(30.3),92.1(33.4),95.6(35.3),97.0(36.1),90.5(32.5),81.8(27.7),71.4(21.9),62.7(17.1),79.8(26.6)
Average low °F (°C),41.5(5.3),44.8(7.1),51.3(10.7),58.6(14.8),66.7(19.3),72.3(22.4),74.4(23.6),74.6(23.7),69.4(20.8),60.6(15.9),50.6(10.3),42.3(5.7),59.0(15.0)
Mean minimum °F (°C),26.7(−2.9),28.5(−1.9),34.7(1.5),42.9(6.1),54.1(12.3),64.2(17.9),69.9(21.1),69.0(20.6),56.9(13.8),44.7(7.1),34.6(1.4),26.7(−2.9),22.0(−5.6)
Record low °F (°C),−2(−19),−1(−18),18(−8),30(−1),40(4),51(11),57(14),58(14),41(5),30(−1),20(−7),4(−16),−2(−19)
Average precipitation inches (mm),2.22(56),2.02(51),2.76(70),2.09(53),4.44(113),4.33(110),1.88(48),2.35(60),2.99(76),3.88(99),2.96(75),2.40(61),34.32(872)
Average snowfall inches (cm),0.4(1.0),0.2(0.51),0(0),0(0),0(0),0(0),0(0),0(0),0(0),0(0),0(0),trace,0.6(1.5)
Average precipitation days (≥ 0.01 in),7.4,7.4,9.2,7.1,8.9,7.7,5.4,4.9,6.7,7.5,7.5,7.8,87.5


In [29]:
climate_data_dict['Freetown']

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Record high °C (°F),36.0(96.8),38.0(100.4),38.0(100.4),38.5(101.3),35.0(95.0),35.0(95.0),33.0(91.4),32.0(89.6),31.0(87.8),35.0(95.0),36.5(97.7),33.5(92.3),38.5(101.3)
Average high °C (°F),29.9(85.8),30.3(86.5),30.9(87.6),31.2(88.2),30.9(87.6),30.1(86.2),28.7(83.7),28.4(83.1),29.0(84.2),29.9(85.8),30.1(86.2),29.7(85.5),29.9(85.9)
Daily mean °C (°F),27.3(81.1),27.6(81.7),28.1(82.6),28.5(83.3),28.3(82.9),27.4(81.3),26.3(79.3),25.9(78.6),26.4(79.5),27.1(80.8),27.7(81.9),27.5(81.5),27.3(81.2)
Average low °C (°F),23.8(74.8),24.0(75.2),24.4(75.9),24.8(76.6),24.4(75.9),23.6(74.5),23.1(73.6),23.0(73.4),23.1(73.6),23.4(74.1),24.0(75.2),24.1(75.4),23.8(74.9)
Record low °C (°F),15.0(59.0),17.8(64.0),19.4(66.9),20.0(68.0),20.6(69.1),20.0(68.0),19.4(66.9),19.4(66.9),20.0(68.0),19.4(66.9),20.0(68.0),15.6(60.1),15.0(59.0)
Average rainfall mm (inches),8.0(0.31),6.0(0.24),28.0(1.10),68.0(2.68),214.0(8.43),522.0(20.55),"1,190(46.85)","1,078(42.44)",800.0(31.50),333.0(13.11),148.0(5.83),38.0(1.50),"4,433(174.54)"
Average rainy days,0,0,1,4,15,22,27,27,24,21,9,2,152
Average relative humidity (%) (at 15:00 LST),66,67,67,68,73,76,81,82,80,78,76,69,74
Mean monthly sunshine hours,226.3,215.6,232.5,207.0,189.1,153.0,102.3,86.8,126.0,186.0,198.0,161.2,"2,083.8"


### Selecting common Wikipedia Cliamte Variables

More general it can be examined how different each table is by considering the number of variables. In the below cell is every cliamte variable for every table exstracted in one long list. By examining the list as set it can be explored just how many different cliamte variable there are.

In [34]:
city_climate_list = [list(city_climate_df.index.values) for city, city_climate_df in climate_data_dict.items() if city_climate_df is not None]
climate_variables = [city for sublist in city_climate_list for city in sublist]
set_of_climate_variable = set(climate_variables)
print('In the cliamte tables there is, {} different names for cliamte variables that is measured.'.format(len(set_of_climate_variable)))

In the cliamte tables there is, 69 different names for cliamte variables that is measured.


Though there are 69 different names for cliamte variables some appear much more frequently and some have different names for the same thing. In the cell below is number of time the variables is seen displayed next the name of the climate variable to indicate which appear the most.

In [35]:
varaible_count = pd.Series(climate_variables).value_counts()
[(count, name) for name, count in zip(varaible_count.index, varaible_count)]

[(298, 'Mean monthly sunshine hours'),
 (270, 'Average low °C (°F)'),
 (270, 'Average high °C (°F)'),
 (254, 'Record low °C (°F)'),
 (254, 'Record high °C (°F)'),
 (252, 'Average relative humidity (%)'),
 (251, 'Daily mean °C (°F)'),
 (191, 'Average precipitation mm (inches)'),
 (102, 'Percent possible sunshine'),
 (85, 'Average rainfall mm (inches)'),
 (78, 'Average ultraviolet index'),
 (62, 'Average rainy days'),
 (57, 'Average snowy days'),
 (56, 'Average low °F (°C)'),
 (56, 'Average high °F (°C)'),
 (56, 'Average precipitation days (≥ 0.1 mm)'),
 (55, 'Record low °F (°C)'),
 (55, 'Record high °F (°C)'),
 (53, 'Mean minimum °F (°C)'),
 (53, 'Mean maximum °F (°C)'),
 (49, 'Average precipitation inches (mm)'),
 (46, 'Mean daily sunshine hours'),
 (45, 'Average precipitation days (≥ 0.01 in)'),
 (43, 'Average precipitation days (≥ 1.0 mm)'),
 (39, 'Average snowfall inches (cm)'),
 (38, 'Average snowy days (≥ 0.1 in)'),
 (33, 'Average dew point °F (°C)'),
 (28, 'Average precipitation 

It appears as the following five variable 

1. `'Mean monthly sunshine hours'`
2. `'Average high °C'`
3. `'Average low °C'`
4. `'Average precipitation mm'`
5. `'Average precipitation days'`

can be extracted accross almost all cliamte tables if one consider multiple names for the above thing and relax the constriant for what is considered a day with precipitation.  Below a dictionary have been created which helps to rename the variable such that they are only called the five above things. 

In the cell below a dictionary have been intialized which maps selected cliamte variable to the 5 mention above. This will help rename the variables in every data frame.

In [32]:
rename_column = {
    'Mean monthly sunshine hours'               : 'Mean monthly sunshine hours',
    'Average high °C (°F)'                      : 'Average high °C',
    'Average high °F (°C)'                      : 'Average high °C',
    'Average low °C (°F)'                       : 'Average low °C',
    'Average low °F (°C)'                       : 'Average low °C',
    'Average rainfall inches (mm)'              : 'Average precipitation mm',
    'Average rainfall mm (inches)'              : 'Average precipitation mm',
    'Average precipitation inches (mm)'         : 'Average precipitation mm',
    'Average precipitation mm (inches)'         : 'Average precipitation mm',
    'Average precipitation mm'                  : 'Average precipitation mm',
    'Average precipitation days'                : 'Average precipitation days',
    'Average precipitation days (≥ 0.005 inch)' : 'Average precipitation days',
    'Average precipitation days (≥ 0.01 in)'    : 'Average precipitation days',
    'Average precipitation days (≥ 0.01 in.)'   : 'Average precipitation days',
    'Average precipitation days (≥ 0.01 inch)'  : 'Average precipitation days',
    'Average precipitation days (≥ 0.01\xa0mm)' : 'Average precipitation days',
    'Average precipitation days (≥ 0.1 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 0.1\xa0mm)'  : 'Average precipitation days',
    'Average precipitation days (≥ 0.2 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 0.2\xa0mm)'  : 'Average precipitation days',
    'Average precipitation days (≥ 0.5 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 1.0 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 1\xa0mm)'    : 'Average precipitation days',
    'Average rainy days'                        : 'Average precipitation days',
    'Average rainy days (≥ 0.01 in)'            : 'Average precipitation days',
    'Average rainy days (≥ 0.01 inch)'          : 'Average precipitation days',
    'Average rainy days (≥ 0.01 mm)'            : 'Average precipitation days',
    'Average rainy days (≥ 0.1 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 0.2 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 0.5 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 1 mm)'               : 'Average precipitation days',
    'Average rainy days (≥ 1.0 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 1.0\xa0mm)'          : 'Average precipitation days',
    'Average rainy days (≥ 1mm)'                : 'Average precipitation days',
}

In the cell below the function `keep_selected_varibles` has been developed that takes the above dictionary as an input as well as dataframe decribing a city climate. The function only returns the values of the keys of the dictionary. That means that only 5 four varaibles will be considered. An example of the function can be seen below with city of Austin.

In [30]:
def keep_selected_varibles(city_climate_df, rename_column):
    
    # Rename remaing climate variables:
    city_climate_variables = city_climate_df.index.tolist()
    name_change  = {climate_variable : rename_column[climate_variable] for climate_variable  in city_climate_variables if climate_variable in rename_column}
    city_climate_df.rename(name_change, inplace = True)

    # only keep the variables where the name was updated:
    city_climate_df_update = city_climate_df.loc[name_change.values()]

    # Remove duplicate indexes...
    # This can happen if a city e.g. Toronto has 'Average precipitation mm (inches)',
    # 'Average rainfall mm (inches)', 'Average snowfall cm (inches)' i.e. precipitation = rainfall + snowfall
    # THis method keeps the first index. Hopefully in all cases this it the total precipitation.
    city_climate_df_update = city_climate_df_update[~city_climate_df_update.index.duplicated(keep='first')]

    return city_climate_df_update

In [33]:
austin_tmp = climate_data_dict['Austin'].copy()
austin_tmp = keep_selected_varibles(austin_tmp, rename_column)
austin_tmp

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Average high °C,61.5(16.4),65.2(18.4),72.2(22.3),79.8(26.6),86.5(30.3),92.1(33.4),95.6(35.3),97.0(36.1),90.5(32.5),81.8(27.7),71.4(21.9),62.7(17.1),79.8(26.6)
Average low °C,41.5(5.3),44.8(7.1),51.3(10.7),58.6(14.8),66.7(19.3),72.3(22.4),74.4(23.6),74.6(23.7),69.4(20.8),60.6(15.9),50.6(10.3),42.3(5.7),59.0(15.0)
Average precipitation mm,2.22(56),2.02(51),2.76(70),2.09(53),4.44(113),4.33(110),1.88(48),2.35(60),2.99(76),3.88(99),2.96(75),2.40(61),34.32(872)
Average precipitation days,7.4,7.4,9.2,7.1,8.9,7.7,5.4,4.9,6.7,7.5,7.5,7.8,87.5
Mean monthly sunshine hours,163.8,169.3,205.9,205.8,227.1,285.5,317.2,297.9,233.8,215.6,168.3,153.5,"2,643.7"


### Selecting metric represenation of Wikipedia Cliamte Variables
In the above dataframe, that shows the selected cliamte data for Austin, there is both the metric and imperial represenation in a string data format. This needs to be processed such that it is numeric format hence there can only be one represenation. The metric one has been selected. To ensure a numeric metric entepreation of the dataframe the following four functions have been created.

1. `select_celcius`: This function split the string e.g. "61.58(16.4)" into two number 61.48 and 16.4 and select the *smallest* of the two. This ensure that metric represenation of the temperature is always selected i.e. celcius. (except in the case when it is colder than -40 °C. But this is no worry for this sort of data).
1. `select_mm`: This function split the string e.g. "2.22(56)" into two number 2.22 and 56 and selects the *largest*. This ensure that metric represenation of the temperature is always selected i.e. minimeters. 
1. `ensure_numeric`: This function enures that the data is formatted as float.
1. `convert_to_metric`:  This function utilized the above functions to convert the entire dataframe into metric and numeric format.


In [39]:
def select_celcius(row):

    matric = []
    for imperial_metric in row:
        try:
            val1, val2 = imperial_metric.replace(")", "").split('(')
            
            # Remove symbol which look like a minus sign which is not not a minus sign
            val1, val2 = val1.replace('−', '-'), val2.replace('−', '-')
            # Remove commas which seperates tree zeros and convert to float 
            val1, val2 = float(val1.replace(',', '')), float(val2.replace(',', ''))

            matric.append(min(val1, val2))
        except ValueError:
            try:
                matric.append(float(imperial_metric))
            except ValueError:
                matric.append(None)
    return matric



def select_mm(row):

    matric = []
    for imperial_metric in row:
        try:
            val1, val2 = imperial_metric.replace(")", "").split('(')
            
            # Remove symbol which look like a minus sign which is not not a minus sign
            val1, val2 = val1.replace('−', '-'), val2.replace('−', '-')
            # Remove commas which seperates tree zeros and convert to float 
            val1, val2 = float(val1.replace(',', '')), float(val2.replace(',', ''))

            matric.append(max(val1, val2))
        except ValueError:
            try:
                matric.append(float(imperial_metric))
            except ValueError:
                matric.append(None)
    return matric


def convert_to_metric(city_climate_df, function_dict):
    
    columns = city_climate_df.columns
    metric_city_climate_df = pd.DataFrame(columns = columns)

    climate_variables = []
    for climate_var, row in city_climate_df.iterrows():
        climate_variables.append(climate_var)
        metric = function_dict[climate_var]( city_climate_df.loc[climate_var] )

        tmp = pd.DataFrame({key: [value] for key, value in zip(columns, metric)})
        metric_city_climate_df = metric_city_climate_df.append(tmp)

    metric_city_climate_df['Climate_variable'] = climate_variables
    metric_city_climate_df.set_index('Climate_variable', inplace = True)

    return metric_city_climate_df

In [40]:
def ensure_numeric(row):
    metric = []
    for val in row:
        try:
            val = float(val.replace(',', ''))
            metric.append(val)
        except ValueError:
            matric.append(np.nan)
    
    return metric

Now all the function are utlized on the data for city Austin. As can be seen below only the metric repreation is left. 

In [42]:
function_dict = {
    'Average high °C' : select_celcius,
    'Average low °C' : select_celcius,
    'Average precipitation mm' : select_mm,
    'Average precipitation days' : ensure_numeric,
    'Mean monthly sunshine hours' : ensure_numeric
}

In [43]:
convert_to_metric(
    city_climate_df = austin_tmp,
    function_dict = function_dict
)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Average high °C,16.4,18.4,22.3,26.6,30.3,33.4,35.3,36.1,32.5,27.7,21.9,17.1,26.6
Average low °C,5.3,7.1,10.7,14.8,19.3,22.4,23.6,23.7,20.8,15.9,10.3,5.7,15.0
Average precipitation mm,56.0,51.0,70.0,53.0,113.0,110.0,48.0,60.0,76.0,99.0,75.0,61.0,872.0
Average precipitation days,7.4,7.4,9.2,7.1,8.9,7.7,5.4,4.9,6.7,7.5,7.5,7.8,87.5
Mean monthly sunshine hours,163.8,169.3,205.9,205.8,227.1,285.5,317.2,297.9,233.8,215.6,168.3,153.5,2643.7


As can be seen all we're left with is the five selected variable in numerical metric representation.

### Putting everything together to process all cities.

Now the selection of cliamte variables and keeping their numeric metric format will be performed on all cities. 

In [44]:
index = pd.Index(
    ['Average high °C', 'Average low °C','Average precipitation mm', 'Average precipitation days', 'Mean monthly sunshine hours'],
    dtype='object', name='Climate_variable')

empty_df = pd.DataFrame({
    'Jan'   : np.repeat(np.nan, 5),
    'Feb'   : np.repeat(np.nan, 5),
    'Mar'   : np.repeat(np.nan, 5),
    'Apr'   : np.repeat(np.nan, 5),
    'May'   : np.repeat(np.nan, 5), 
    'Jun'   : np.repeat(np.nan, 5),
    'Jul'   : np.repeat(np.nan, 5),
    'Aug'   : np.repeat(np.nan, 5),
    'Sep'   : np.repeat(np.nan, 5),
    'Oct'   : np.repeat(np.nan, 5),
    'Nov'   : np.repeat(np.nan, 5),
    'Dec'   : np.repeat(np.nan, 5),
    'Year'  : np.repeat(np.nan, 5),
}, index = index)

cleaned_city_dict = dict()
for i, (city, city_climate_df) in enumerate(climate_data_dict.items()):

    if city_climate_df is None:
        # Add empty DF in case the city does not have climate table.
        cleaned_city_dict[city] = empty_df
        continue

    # Keep only selected columns and remname them:
    city_climate_df_update = keep_selected_varibles(
        city_climate_df = city_climate_df,
        rename_column = rename_column
    )

    # extract the metric representaiton
    metric_city_climate_df = convert_to_metric(
        city_climate_df = city_climate_df_update,
        function_dict = function_dict
    )

    # Add missing values
    metric_city_climate_df = metric_city_climate_df.append(empty_df)
    metric_city_climate_df = metric_city_climate_df[~metric_city_climate_df.index.duplicated(keep='first')]
    metric_city_climate_df = metric_city_climate_df.reindex(index)


    # Update the city dataframe such that it only keeps the choosen climate variables
    # and they are metric
    cleaned_city_dict[city] = metric_city_climate_df

#### Pivot the data for every city in one dataframe
Now every cliamte dataframe will be put togeter to form a big dataframe which contains the cliamte data for every city. 

In [49]:
def stack_climate_data(city_climate_df):
    stacked_city_climate_date = city_climate_df.stack().to_frame()
    stacked_city_climate_date.index = stacked_city_climate_date.index.map('_'.join)

    return stacked_city_climate_date.T


stacked_city_climate_date = cleaned_city_dict['Freetown'].stack().to_frame()
stacked_city_climate_date.index = stacked_city_climate_date.index.map('_'.join)


for i, city in enumerate(cleaned_city_dict):
    city_df = pd.DataFrame({
        'wiki_link' : [city]
    })
    climate_data = stack_climate_data(cleaned_city_dict[city])

    city_cliamte_df = pd.concat([city_df, climate_data], axis=1, join='inner')

    if i == 0:
        all_cities_climate_df = city_cliamte_df

    else:
        all_cities_climate_df = all_cities_climate_df.append(city_cliamte_df)


final_data = pd.merge(df_geomet, all_cities_climate_df, how = 'inner').drop(columns=["Latitude", "Longitude"])

## Imputing the final data
Now all that is missing is imputing the missing data such that the data can be used for modelling. The imputing will be caried out with closest geographical neighbour scheme used preivosuly 

In [50]:
final_data

,City,Country,Dist_Equator,Elevation,Dist_to_coast,wiki_link,Average high °C_Jan,Average high °C_Feb,Average high °C_Mar,Average high °C_Apr,...,Mean monthly sunshine hours_Apr,Mean monthly sunshine hours_May,Mean monthly sunshine hours_Jun,Mean monthly sunshine hours_Jul,Mean monthly sunshine hours_Aug,Mean monthly sunshine hours_Sep,Mean monthly sunshine hours_Oct,Mean monthly sunshine hours_Nov,Mean monthly sunshine hours_Dec,Mean monthly sunshine hours_Year
0,Baltimore(MD),United States,4368.95,6,2,Baltimore,5.8,7.6,12.2,18.6,...,230.7,254.5,277.3,290.1,264.4,221.8,205.5,158.5,144.5,2581.7
1,Melbourne,Australia,4204.75,25,4,Melbourne,27.0,26.9,24.6,21.1,...,186.0,142.6,123.0,136.4,167.4,186.0,226.3,225.0,263.5,2384.1
2,Niamey,Niger,1503.89,209,781,Niamey,32.5,35.7,39.1,40.9,...,251.0,257.0,251.0,238.0,203.0,228.0,285.0,285.0,276.0,3082.0
3,Hanoi,Vietnam,2338.37,19,77,Hanoi,19.7,20.1,22.9,27.2,...,90.0,183.0,172.0,195.0,174.0,176.0,167.0,137.0,124.0,1586.0
4,Urumqi,China,4828.06,2071,2312,Urumqi,-7.4,-4.7,2.7,16.1,...,248.0,283.3,282.7,298.7,301.0,262.6,224.4,127.4,84.3,2523.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,Durban,South Africa,3320.48,13,2,Durban,27.8,28.0,27.7,26.1,...,206.4,223.6,224.9,230.4,217.0,173.3,169.4,166.1,189.9,2365.4
327,Kumasi,Ghana,744.793,261,177,Kumasi,31.9,33.5,32.9,32.3,...,204.0,204.7,146.3,101.2,77.0,106.2,161.4,193.8,178.0,1951.8
328,Izmir,Turkey,4271.52,126,1,Izmir,12.3,13.6,16.2,20.9,...,237.0,303.8,345.0,378.2,368.9,303.0,232.5,165.0,127.1,2931.9
329,Ankara,Turkey,4416.22,1069,197,Ankara,4.2,6.4,11.5,17.4,...,195.0,263.5,303.0,353.4,334.8,276.0,207.7,138.0,77.5,2504.0


In [51]:
final_data.to_csv("../../data/processed/GeoMet.csv")